<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/p_tuning_on_biomed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9

In [ ]:
from datasets import load_dataset

dataset_name = "Shushant/BiomedicalQuestionAnsweringDataset"
dataset = load_dataset(dataset_name, "train")
dataset = dataset["train"].train_test_split(test_size=2000, train_size=8000)

In [ ]:
import ast
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

def merge_answer(record):
  # convert string answer to dictionary
  answer = ast.literal_eval(record["answers"])["text"]
  record["answers"] = answer
  return record

# load T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
# the context summarization will be using T5 as explored previously
summarizer = AutoModelWithLMHead.from_pretrained('t5-small', return_dict=True)

def summarize_large_context(record):
  # the max_seq_len is 512, longer than that it will be automatically cut off
  if len(torch.split(record["context"], " ")) > 512:
    input_seq = tokenizer.encode("sumarize: " + record["context"], return_tensors="pt", max_length=512, truncation=True)
    summarized_seq = model.generate(input_seq, min_length = 100, max_length = 512)
    record["context"] = tokenizer.decode(summarized_seq[0])
  return record

def generate_text_field(record):
  record['text'] = "### Context: " + record['context'] + "### Question: " + record['question'] + "### Answer: " + record['answers']
  return record

training_dataset = dataset["train"]

training_dataset.add_column(name="text", column=["temp"]*len(training_dataset))

training_dataset = training_dataset.map(merge_answer)
training_dataset = training_dataset.map(summarize_large_context)
training_dataset = training_dataset.map(generate_text_field, remove_columns=training_dataset.column_names)

In [ ]:
eval_dataset = dataset["test"]
eval_dataset.

In [ ]:
from peft import PromptEncoderConfig

#Use LoRA config for fine-tuning this model
peft_config = PromptEncoderConfig(task_type="CAUSAL_LM", num_virtual_tokens=20, encoder_hidden_size=128)



In [ ]:
import torch
from transformers import  AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
model_name = "cosmin/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)
model.config.use_cache = False


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3 # based on qlora paper
max_steps = 300
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to=None
)


In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()
